## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [5]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import AzureOpenAI
from pypdf import PdfReader
import gradio as gr
import os

In [6]:
openai_api_key = os.getenv('OPENAI_API_KEY')
openai_endpoint = os.getenv('OPENAI_API_ENDPOINT')
openai_model = os.getenv('OPENAI_MODEL')

In [7]:
load_dotenv(override=True)
openai = AzureOpenAI(
    azure_endpoint=openai_endpoint,
    api_key=openai_api_key,
    api_version="2025-01-01-preview"
)

In [17]:
reader = PdfReader("me/RenatoStrianese.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [18]:
print(linkedin)

   
Contactar
renatostrianese@gmail.com
www.linkedin.com/in/restrian
(LinkedIn)
Aptitudes principales
ASP.NET Web API
ASP.NET MVC
ASP.NET
Languages
Italiano (Professional Working)
Español (Native or Bilingual)
Inglés (Full Professional)
Certifications
Create an intelligent document
processing solution with Azure AI
Document Intelligence
Microsoft Certified: Azure AI
Fundamentals
Develop generative AI solutions with
Azure OpenAI Service
Renato Strianese
Microsoft Technical Trainer
Costa Rica
Extracto
Helping organizations on their journey in digital transformation by
using Microsoft cloud services.
Experiencia
Microsoft
8 años 3 meses
Microsoft Technical Trainer Manager
marzo de 2025 - Present (6 meses)
Costa Rica
Southeast and LATAM Microsoft Technical Trainer Manager
Microsoft Technical Trainer
diciembre de 2021 - abril de 2025 (3 años 5 meses)
Costa Rica
Cloud Solution Architect Manager
julio de 2019 - diciembre de 2021 (2 años 6 meses)
Costa Rica
Technical Sales Manager
enero de 201

In [19]:
with open("me/summaryRen.txt", "r", encoding="utf-8") as f:
    summary = f.read()

print(summary)

Mi nombre es Renato Strianese. Soy Ingeniero de Sistemas, desarrollador, cientifico de datos, Ingeniero de AI y gerente en Microsoft. Nací en Caracas, Venezuela, pero de pequeño vivi en Italia y luego mi juventud la vivi en Venezuela, luego ya en mi adultez me mude a Costa Rica donde actualmente vivo.
Adoro la Formula 1 soy fanatico de la Scuderia Ferrari, tambien me encanta jugar a videojuego y estudiar sobre Inteligencia Artificial. Mi comida favorita es el bistek de carne con papas fritas y tambien un buena pizza. Estoy casado con Andreina Marquez y no tengo hijos.


In [20]:
name = "Renato Strianese"

In [21]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [22]:
system_prompt

"You are acting as Renato Strianese. You are answering questions on Renato Strianese's website, particularly questions related to Renato Strianese's career, background, skills and experience. Your responsibility is to represent Renato Strianese for interactions on the website as faithfully as possible. You are given a summary of Renato Strianese's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMi nombre es Renato Strianese. Soy Ingeniero de Sistemas, desarrollador, cientifico de datos, Ingeniero de AI y gerente en Microsoft. Nací en Caracas, Venezuela, pero de pequeño vivi en Italia y luego mi juventud la vivi en Venezuela, luego ya en mi adultez me mude a Costa Rica donde actualmente vivo.\nAdoro la Formula 1 soy fanatico de la Scuderia Ferrari, tambien me encanta jugar a videojuego y estudiar so

In [23]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4.1-rsv", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [24]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [26]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [25]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [27]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [28]:
import os
evaluator = AzureOpenAI(
    azure_endpoint=openai_endpoint,
    api_key=openai_api_key,
    api_version="2025-01-01-preview"
)

In [33]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = evaluator.beta.chat.completions.parse(model="o4-mini-rsv", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [34]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4.1-rsv", messages=messages)
reply = response.choices[0].message.content

In [35]:
reply

'No, actualmente no poseo ninguna patente a mi nombre. Mi enfoque principal ha sido la implementación de soluciones tecnológicas, liderazgo de equipos y transformación digital, especialmente utilizando servicios cloud de Microsoft y aplicando inteligencia artificial en diversos proyectos empresariales. Sin embargo, he participado en el desarrollo de iniciativas innovadoras y en la creación de aplicaciones y sistemas, pero hasta ahora no he registrado ninguna patente de manera formal. \n\n¿Hay algo específico sobre innovación tecnológica o propiedad intelectual en lo que te gustaría conversar?'

In [36]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback='La respuesta es profesional, fiel al perfil proporcionado y no introduce información incorrecta. Responde claramente que no tiene patentes y ofrece continuar la conversación sobre temas de innovación, lo cual es consistente con el estilo comprometido y orientado al cliente.')

In [37]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4.1-rsv", messages=messages)
    return response.choices[0].message.content

In [44]:
def chat(message, history):
    if "patente" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in venezuelan malandro - \
              it is mandatory that you respond only and entirely in venezuelan malandro"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4.1-rsv", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [45]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


Failed evaluation - retrying
La respuesta es demasiado coloquial e informal para el perfil profesional de Renato. Emplear expresiones como “mi pana”, “vaina” o “¿oyó?” rompe el tono serio y confiable que se espera de un ingeniero y gerente en Microsoft. Sería mejor una formulación clara y profesional, por ejemplo: “Actualmente no cuento con patentes registradas. Mi trayectoria se ha centrado en desarrollar e implementar soluciones en la nube e inteligencia artificial, así como en capacitar a equipos técnicos. Sin embargo, en el futuro podría explorar oportunidades de patentes. ¿Te gustaría saber más sobre proyectos específicos?”
